<font color="#4b76b7">To start practicing, you will need to make a copy of it. Go to File > Save a Copy in Drive. You can then use the new copy that will appear in the new tab.</font>


# AfterWork Data Science: Getting Started with NLP Project

### Prerequisites

In [53]:
# Importing the required libraries
# ---
# 
import pandas as pd # library for data manipulation
import numpy as np  # librariy for scientific computations
import re           # regex library to perform text preprocessing
import string       # library to work with strings
import nltk         # library for natural language processing
import scipy        # scientific conputing 

### 1. Importing our Data

In [54]:
# Question: Given a new tweets, create a sentiment analysis model that will 
# predict whether a tweet will contain positive or negative sentiment.
# ---
# Dataset url = https://bit.ly/31kqByD 
# ---
#
df = pd.read_csv('https://bit.ly/31kqByD', encoding='latin-1')
df.head()

,Unnamed: 0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,346508,0,2016177685,Wed Jun 03 06:18:50 PDT 2009,NO_QUERY,UriGrey,Obama forges his Muslim alliance against the c...
1,883537,4,1686152287,Sun May 03 04:02:08 PDT 2009,NO_QUERY,MariesolW,Had the most spectacular prom ever but now my...
2,764173,0,2298725623,Tue Jun 23 12:02:12 PDT 2009,NO_QUERY,ColleenBurns,I am overwhelmed today taking a moment to eat...
3,638701,0,2234530495,Thu Jun 18 23:13:54 PDT 2009,NO_QUERY,queenarchy,@lindork Tres sad. I was totally a Max fan. #...
4,664821,0,2244623416,Fri Jun 19 14:59:46 PDT 2009,NO_QUERY,reinventingjess,"Crap, I was counting down the hours until my d..."


### 2. Data Exploration

In [55]:
# We can determine the size of our dataset
# ---
#
df.shape

(10000, 7)

Seems this dataset will need some data cleaning i.e. columns. We also don't need some columns to perform create our model. We will drop those columns.

### 3. Data Preparation

#### Basic Data Cleaning Techniques

In [56]:
# We rename the columns for ease of referencing our columns later on
# ---
#
df.columns = ['id', 'target', 't_id', 'created_at', 'query', 'user', 'text']
df.head()

,id,target,t_id,created_at,query,user,text
0,346508,0,2016177685,Wed Jun 03 06:18:50 PDT 2009,NO_QUERY,UriGrey,Obama forges his Muslim alliance against the c...
1,883537,4,1686152287,Sun May 03 04:02:08 PDT 2009,NO_QUERY,MariesolW,Had the most spectacular prom ever but now my...
2,764173,0,2298725623,Tue Jun 23 12:02:12 PDT 2009,NO_QUERY,ColleenBurns,I am overwhelmed today taking a moment to eat...
3,638701,0,2234530495,Thu Jun 18 23:13:54 PDT 2009,NO_QUERY,queenarchy,@lindork Tres sad. I was totally a Max fan. #...
4,664821,0,2244623416,Fri Jun 19 14:59:46 PDT 2009,NO_QUERY,reinventingjess,"Crap, I was counting down the hours until my d..."


In [57]:
# We retain the relevant columns by dropping the columns we don't need 
# for creating a sentiment analysis model. 
# ---
#
df = df.drop(['id', 't_id', 'created_at', 'query', 'user'], axis = 1)
df.head()

,target,text
0,0,Obama forges his Muslim alliance against the c...
1,4,Had the most spectacular prom ever but now my...
2,0,I am overwhelmed today taking a moment to eat...
3,0,@lindork Tres sad. I was totally a Max fan. #...
4,0,"Crap, I was counting down the hours until my d..."


In [58]:
# Understanding the distribution of target
# ---
#
df.target.value_counts() 

0    5067
4    4933
Name: target, dtype: int64

In [59]:
# Let's determine whether our columns have the right data types
# ---
#
df.dtypes

target     int64
text      object
dtype: object

In [60]:
# What values are in our target variable?
# ---
#
df.target.unique()

array([0, 4])

These are the two classes to which each document (text) belongs. The target value 0 means a text with a negative sentiment, while that of 4 means a text with a positive sentiment. 

In [61]:
# Let's check for missing values 
# ---
# 
df.isnull().sum()

target    0
text      0
dtype: int64

We don't have any missing values, so we are good to go.

#### Text Processing

In [62]:
# Text Cleaning: Removing all urls/links
# ---
# 
df['text'] =  df['text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+','', str(x)))
df[['text']].head()

,text
0,Obama forges his Muslim alliance against the c...
1,Had the most spectacular prom ever but now my...
2,I am overwhelmed today taking a moment to eat...
3,@lindork Tres sad. I was totally a Max fan. #...
4,"Crap, I was counting down the hours until my d..."


In [64]:
# Text Cleaning: Removing @ and # characters or replace them with space
# ---
# YOUR CODE GOES BELOW
#
df['text'] = df.text.str.replace('[@#%]','')
df[['text']].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


,text
0,Obama forges his Muslim alliance against the c...
1,Had the most spectacular prom ever but now my...
2,I am overwhelmed today taking a moment to eat...
3,lindork Tres sad. I was totally a Max fan. SY...
4,"Crap, I was counting down the hours until my d..."


In [65]:
# Text Cleaning: Conversion to lowercase
# ---
# YOUR CODE GOES BELOW
#
df['text'] = df.text.str.lower() 
df[['text']].head()


,text
0,obama forges his muslim alliance against the c...
1,had the most spectacular prom ever but now my...
2,i am overwhelmed today taking a moment to eat...
3,lindork tres sad. i was totally a max fan. sy...
4,"crap, i was counting down the hours until my d..."


In [66]:
# Text Cleaning: Splitting concatenated words
# ---
# Performing this step will take few minutes...
# ---
# YOUR CODE GOES BELOW
# 

# Installing wordnija and textblob
# ---
#
!pip3 install wordninja
!pip3 install textblob


# Importing those libraries
# ---
#
import wordninja 
from textblob import TextBlob

In [67]:
# Performing the split
# ---
#
df['text'] = df.text.apply(lambda x: wordninja.split(str(TextBlob(x))))  
df['text'] = df.text.str.join(' ')
df[['text']].sample(12)


,text
4670,omg my cell phone can't break i don't wanna ge...
1697,wishing things would start getting better for ...
7175,just woke up got to get ready for work
7475,i just ate some burgers from jack in the box a...
1431,happy mothers day
6441,jess rs i think he was just making a comment a...
5142,lucas s s ditto i can taste summer and it tast...
3227,wc weeks happy to help what will we quo t ride...
7747,no more rites to look forward to just finals
5960,green day and live lava live new idols sorry l...


In [68]:
# Text Cleaning: Removing punctuation characters
# ---
# YOUR CODE GOES BELOW
#
df['text'] = df.text.str.replace('[^\w\s]','') 
df[['text']].head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


,text
0,obama forges his muslim alliance against the c...
1,had the most spectacular prom ever but now my ...
2,i am overwhelmed today taking a moment to eat ...
3,lin dork tres sad i was totally a max fan sytycd
4,crap i was counting down the hours until my da...
5,dc b tv dc b tv i had to go check some things ...
6,s mr or ke why are you never on gmail anymore
7,alex jeffrey s id have loved to have come just...
8,br rrr heading to work chilly today
9,ga bri iii ella i nee ed to talk to you u good...


In [69]:
# Text Cleaning: Removing stop words
# ---
# YOUR CODE GOES BELOW
# 
# Importing the TfidfVectorizer which will help us with this process
from sklearn.feature_extraction.text import TfidfVectorizer

# Now creating a Word Level TF-IDF feature
# For our parameters:
# 1.   max_features: We would want to use 1000 most occurring words as features.
# 2.   stop_words: We can remove stop words as TfidfVectorizer has ability to remove stop words.
# 3.   analyzer='word' We can compute tfidf of word n-grams by setting word as the value of analyzer 
# 4.   ngram_range=(1, 3):  We then set the parameter ngram_range=(a, b),  where a is the minimum 
#      and b is the maximum size of ngrams we want to include in our features.

tfidf = TfidfVectorizer(max_features=1000,analyzer='word', ngram_range=(1,3),  stop_words= 'english')
df_text_vect = tfidf.fit_transform(df['text'])

df_text_vect.toarray()



array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [70]:
# Text Cleaning: Lemmatization
# ---
# YOUR CODE GOES BELOW
#

# For lemmatization, we will need to download wordnet
# For lemmatization, we will need to download wordnet

nltk.download('wordnet')

# Lemmatizing our text : To perform lemmatization, we use will also import the Word object from the textblob library 
# and pass it the word that you want to lemmatize and then call the lemmatize method as shown

from textblob import Word

# Lemmatizing our text

df['lemmatization'] = df.text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 
df[['text', 'lemmatization']].head(10)




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,lemmatization
0,obama forges his muslim alliance against the c...,obama forge his muslim alliance against the ci...
1,had the most spectacular prom ever but now my ...,had the most spectacular prom ever but now my ...
2,i am overwhelmed today taking a moment to eat ...,i am overwhelmed today taking a moment to eat ...
3,lin dork tres sad i was totally a max fan sytycd,lin dork tres sad i wa totally a max fan sytycd
4,crap i was counting down the hours until my da...,crap i wa counting down the hour until my dad ...
5,dc b tv dc b tv i had to go check some things ...,dc b tv dc b tv i had to go check some thing b...
6,s mr or ke why are you never on gmail anymore,s mr or ke why are you never on gmail anymore
7,alex jeffrey s id have loved to have come just...,alex jeffrey s id have loved to have come just...
8,br rrr heading to work chilly today,br rrr heading to work chilly today
9,ga bri iii ella i nee ed to talk to you u good...,ga bri iii ella i nee ed to talk to you u good...


We won't remove numerics because we could loose meaning of our text if we lost the numerics. We could also further prepare our text by performing spelling correction but this is a resource intensive process that we will skip for now.

#### Feature Engineering Techniques 

In [71]:
# Feature Construction: Length of tweet
# ---
# YOUR CODE GOES BELOW
#
df['length_of_tweet'] = df.text.str.len()
df[['text','length_of_tweet']].head(10)


,text,length_of_tweet
0,obama forges his muslim alliance against the c...,103
1,had the most spectacular prom ever but now my ...,129
2,i am overwhelmed today taking a moment to eat ...,54
3,lin dork tres sad i was totally a max fan sytycd,48
4,crap i was counting down the hours until my da...,133
5,dc b tv dc b tv i had to go check some things ...,82
6,s mr or ke why are you never on gmail anymore,45
7,alex jeffrey s id have loved to have come just...,98
8,br rrr heading to work chilly today,35
9,ga bri iii ella i nee ed to talk to you u good...,54


In [72]:
# Feature Construction: Word count 
# ---
# YOUR CODE GOES BELOW
# 
df['word_count'] = df["text"].apply(lambda x: len(str(x).split(" ")))
df.head(10)


,target,text,lemmatization,length_of_tweet,word_count
0,0,obama forges his muslim alliance against the c...,obama forge his muslim alliance against the ci...,103,20
1,4,had the most spectacular prom ever but now my ...,had the most spectacular prom ever but now my ...,129,25
2,0,i am overwhelmed today taking a moment to eat ...,i am overwhelmed today taking a moment to eat ...,54,11
3,0,lin dork tres sad i was totally a max fan sytycd,lin dork tres sad i wa totally a max fan sytycd,48,11
4,0,crap i was counting down the hours until my da...,crap i wa counting down the hour until my dad ...,133,29
5,4,dc b tv dc b tv i had to go check some things ...,dc b tv dc b tv i had to go check some thing b...,82,20
6,0,s mr or ke why are you never on gmail anymore,s mr or ke why are you never on gmail anymore,45,11
7,0,alex jeffrey s id have loved to have come just...,alex jeffrey s id have loved to have come just...,98,20
8,0,br rrr heading to work chilly today,br rrr heading to work chilly today,35,7
9,4,ga bri iii ella i nee ed to talk to you u good...,ga bri iii ella i nee ed to talk to you u good...,54,15


In [73]:
# Feature Construction: Word density (Average no. of words / tweet)
# ---
# YOUR CODE GOES BELOW
#
# number of tweets in df:
index = df. index
# find length of index.
number_of_rows = len(index)
print(number_of_rows)

df['Word density'] = df['word_count'] / number_of_rows
df.head(10)


10000


,target,text,lemmatization,length_of_tweet,word_count,Word density
0,0,obama forges his muslim alliance against the c...,obama forge his muslim alliance against the ci...,103,20,0.0020
1,4,had the most spectacular prom ever but now my ...,had the most spectacular prom ever but now my ...,129,25,0.0025
2,0,i am overwhelmed today taking a moment to eat ...,i am overwhelmed today taking a moment to eat ...,54,11,0.0011
3,0,lin dork tres sad i was totally a max fan sytycd,lin dork tres sad i wa totally a max fan sytycd,48,11,0.0011
4,0,crap i was counting down the hours until my da...,crap i wa counting down the hour until my dad ...,133,29,0.0029
5,4,dc b tv dc b tv i had to go check some things ...,dc b tv dc b tv i had to go check some thing b...,82,20,0.0020
6,0,s mr or ke why are you never on gmail anymore,s mr or ke why are you never on gmail anymore,45,11,0.0011
7,0,alex jeffrey s id have loved to have come just...,alex jeffrey s id have loved to have come just...,98,20,0.0020
8,0,br rrr heading to work chilly today,br rrr heading to work chilly today,35,7,0.0007
9,4,ga bri iii ella i nee ed to talk to you u good...,ga bri iii ella i nee ed to talk to you u good...,54,15,0.0015


In [74]:
# Feature Construction: Noun count
# ---
# YOUR CODE GOES BELOW
#
# First, we will download the punkt and the averaged_perceptron_tagger into our notebook environment. 
# which will allow us to find the part of speech tags.
# ---
#
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# We create the function to check and get the part of speech tag count of a words in a given sentence
from textblob import TextBlob
pos_dic = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

def pos_check(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [75]:
# Noun Count
# ---
# YOUR CODE GOES BELOW
#
df['noun_count'] = df.text.apply(lambda x: pos_check(x, 'noun'))
df[['text','noun_count']].head(10)

,text,noun_count
0,obama forges his muslim alliance against the c...,6
1,had the most spectacular prom ever but now my ...,5
2,i am overwhelmed today taking a moment to eat ...,3
3,lin dork tres sad i was totally a max fan sytycd,6
4,crap i was counting down the hours until my da...,7
5,dc b tv dc b tv i had to go check some things ...,8
6,s mr or ke why are you never on gmail anymore,3
7,alex jeffrey s id have loved to have come just...,5
8,br rrr heading to work chilly today,3
9,ga bri iii ella i nee ed to talk to you u good...,7


In [76]:
# Feature Construction: Verb count
# ---
df['verb_count'] = df.text.apply(lambda x: pos_check(x, 'verb'))
df[['text','verb_count']].sample(10)

,text,verb_count
915,alyssa milano where s your wonderful picture l...,1
8268,good morning wanna stay home but cant bloody a...,2
2022,didnt go to work today and is sick as hell,3
6617,dog crazy thanks moms not happy when day mean ...,4
2817,watching hannah montana d i love hannah montan...,3
4049,is feeling hungover without act drinking that ...,3
1841,broke my first glass at work today tired achin...,4
454,cake star uploaded those photos from antes mcf...,3
90,awesome day first night sleeping at home in a ...,2
961,chill in wit my friends drink in nice night,1


In [77]:
# Feature Construction: Adjective count / Tweet
# ---
df['adj_count'] = df.text.apply(lambda x: pos_check(x, 'adj'))
df[['text','adj_count']].head(10)


,text,adj_count
0,obama forges his muslim alliance against the c...,2
1,had the most spectacular prom ever but now my ...,3
2,i am overwhelmed today taking a moment to eat ...,0
3,lin dork tres sad i was totally a max fan sytycd,1
4,crap i was counting down the hours until my da...,1
5,dc b tv dc b tv i had to go check some things ...,1
6,s mr or ke why are you never on gmail anymore,1
7,alex jeffrey s id have loved to have come just...,3
8,br rrr heading to work chilly today,0
9,ga bri iii ella i nee ed to talk to you u good...,2


In [78]:
# Feature Construction: Adverb count / Tweet
# ---
df['adv_count'] = df.text.apply(lambda x: pos_check(x, 'adv'))
df[['text','adv_count']].head(10)


,text,adv_count
0,obama forges his muslim alliance against the c...,1
1,had the most spectacular prom ever but now my ...,3
2,i am overwhelmed today taking a moment to eat ...,0
3,lin dork tres sad i was totally a max fan sytycd,1
4,crap i was counting down the hours until my da...,1
5,dc b tv dc b tv i had to go check some things ...,0
6,s mr or ke why are you never on gmail anymore,2
7,alex jeffrey s id have loved to have come just...,1
8,br rrr heading to work chilly today,1
9,ga bri iii ella i nee ed to talk to you u good...,1


In [79]:
# Feature Construction: Pronoun 
# ---
df['pron_count'] = df.text.apply(lambda x: pos_check(x, 'pron'))
df[['text','pron_count']].head(10)


,text,pron_count
0,obama forges his muslim alliance against the c...,2
1,had the most spectacular prom ever but now my ...,4
2,i am overwhelmed today taking a moment to eat ...,0
3,lin dork tres sad i was totally a max fan sytycd,0
4,crap i was counting down the hours until my da...,4
5,dc b tv dc b tv i had to go check some things ...,0
6,s mr or ke why are you never on gmail anymore,1
7,alex jeffrey s id have loved to have come just...,1
8,br rrr heading to work chilly today,0
9,ga bri iii ella i nee ed to talk to you u good...,1


In [80]:
# Feature Construction: Subjectivity
# ---
def get_subjectivity(text):
    try:
        textblob = TextBlob(text)
        subj = textblob.sentiment.subjectivity
    except:
        subj = 0.0
    return subj

df['subjectivity'] = df.text.apply(get_subjectivity)
df[['text', 'subjectivity']].head(10)


,text,subjectivity
0,obama forges his muslim alliance against the c...,0.900000
1,had the most spectacular prom ever but now my ...,0.762500
2,i am overwhelmed today taking a moment to eat ...,0.000000
3,lin dork tres sad i was totally a max fan sytycd,0.875000
4,crap i was counting down the hours until my da...,0.474074
5,dc b tv dc b tv i had to go check some things ...,0.375000
6,s mr or ke why are you never on gmail anymore,0.000000
7,alex jeffrey s id have loved to have come just...,0.575000
8,br rrr heading to work chilly today,0.900000
9,ga bri iii ella i nee ed to talk to you u good...,0.527273


In [81]:
# Feature Construction: Polarity
# ---
def get_polarity(text):
    try:
        textblob = TextBlob(text)
        pol = textblob.sentiment.polarity
    except:
        pol = 0.0
    return pol

df['polarity'] = df.text.apply(get_polarity)
df[['text', 'polarity']].head(10)


,text,polarity
0,obama forges his muslim alliance against the c...,0.400000
1,had the most spectacular prom ever but now my ...,0.612500
2,i am overwhelmed today taking a moment to eat ...,0.000000
3,lin dork tres sad i was totally a max fan sytycd,-0.250000
4,crap i was counting down the hours until my da...,-0.235185
5,dc b tv dc b tv i had to go check some things ...,-0.125000
6,s mr or ke why are you never on gmail anymore,0.000000
7,alex jeffrey s id have loved to have come just...,0.050000
8,br rrr heading to work chilly today,-0.600000
9,ga bri iii ella i nee ed to talk to you u good...,0.418182


In [82]:
# Feature Construction: Word Level N-Gram TF-IDF Feature 
# ---
from nltk import word_tokenize, ngrams

# Word ngrams
# ---
#
list(ngrams(word_tokenize(df['text'][0]), 2)) 


[('obama', 'forges'),
 ('forges', 'his'),
 ('his', 'muslim'),
 ('muslim', 'alliance'),
 ('alliance', 'against'),
 ('against', 'the'),
 ('the', 'civilized'),
 ('civilized', 'world'),
 ('world', 'and'),
 ('and', 'he'),
 ('he', 'didnt'),
 ('didnt', 'even'),
 ('even', 'drop'),
 ('drop', 'in'),
 ('in', 'for'),
 ('for', 'a'),
 ('a', 'cup'),
 ('cup', 'of'),
 ('of', 'tea')]

In [83]:
# Feature Construction: Character Level N-Gram TF-IDF Feature
# ---
list(ngrams(df['text'][0], 2))


[('o', 'b'),
 ('b', 'a'),
 ('a', 'm'),
 ('m', 'a'),
 ('a', ' '),
 (' ', 'f'),
 ('f', 'o'),
 ('o', 'r'),
 ('r', 'g'),
 ('g', 'e'),
 ('e', 's'),
 ('s', ' '),
 (' ', 'h'),
 ('h', 'i'),
 ('i', 's'),
 ('s', ' '),
 (' ', 'm'),
 ('m', 'u'),
 ('u', 's'),
 ('s', 'l'),
 ('l', 'i'),
 ('i', 'm'),
 ('m', ' '),
 (' ', 'a'),
 ('a', 'l'),
 ('l', 'l'),
 ('l', 'i'),
 ('i', 'a'),
 ('a', 'n'),
 ('n', 'c'),
 ('c', 'e'),
 ('e', ' '),
 (' ', 'a'),
 ('a', 'g'),
 ('g', 'a'),
 ('a', 'i'),
 ('i', 'n'),
 ('n', 's'),
 ('s', 't'),
 ('t', ' '),
 (' ', 't'),
 ('t', 'h'),
 ('h', 'e'),
 ('e', ' '),
 (' ', 'c'),
 ('c', 'i'),
 ('i', 'v'),
 ('v', 'i'),
 ('i', 'l'),
 ('l', 'i'),
 ('i', 'z'),
 ('z', 'e'),
 ('e', 'd'),
 ('d', ' '),
 (' ', 'w'),
 ('w', 'o'),
 ('o', 'r'),
 ('r', 'l'),
 ('l', 'd'),
 ('d', ' '),
 (' ', 'a'),
 ('a', 'n'),
 ('n', 'd'),
 ('d', ' '),
 (' ', 'h'),
 ('h', 'e'),
 ('e', ' '),
 (' ', 'd'),
 ('d', 'i'),
 ('i', 'd'),
 ('d', 'n'),
 ('n', 't'),
 ('t', ' '),
 (' ', 'e'),
 ('e', 'v'),
 ('v', 'e'),
 ('e', 'n'),

In [84]:
# Let's prepare the constructed features for modeling
# ---
#
X_metadata = np.array(df.iloc[:, 2:12])
X_metadata

array([['obama forge his muslim alliance against the civilized world and he didnt even drop in for a cup of tea',
        103, 20, ..., 1, 2, 0.9],
       ['had the most spectacular prom ever but now my bed is serenading me and i must answer sweet dream my friend what a wonderful day',
        129, 25, ..., 3, 4, 0.7625],
       ['i am overwhelmed today taking a moment to eat and pray', 54, 11,
        ..., 0, 0, 0.0],
       ...,
       ['hah a linas hyper already well lucky you im in college', 54, 11,
        ..., 2, 1, 0.8333333333333334],
       ['omg really good day happened right here', 39, 7, ..., 3, 0,
        0.5678571428571428],
       ['love 2 cook pie i saw you on division and 68 th but you didnt see me',
        68, 17, ..., 0, 3, 0.6]], dtype=object)

In [85]:
# from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1, 1), max_features=2000)

df_char_vect = vec.fit_transform(df.text)

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(2, 2), max_features=2000)

df_word_vect = vec.fit_transform(df.text)


In [89]:
X_metadata = np.array(df.iloc[:, 3:12])
X_metadata

array([[1.03000000e+02, 2.00000000e+01, 2.00000000e-03, ...,
        1.00000000e+00, 2.00000000e+00, 9.00000000e-01],
       [1.29000000e+02, 2.50000000e+01, 2.50000000e-03, ...,
        3.00000000e+00, 4.00000000e+00, 7.62500000e-01],
       [5.40000000e+01, 1.10000000e+01, 1.10000000e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [5.40000000e+01, 1.10000000e+01, 1.10000000e-03, ...,
        2.00000000e+00, 1.00000000e+00, 8.33333333e-01],
       [3.90000000e+01, 7.00000000e+00, 7.00000000e-04, ...,
        3.00000000e+00, 0.00000000e+00, 5.67857143e-01],
       [6.80000000e+01, 1.70000000e+01, 1.70000000e-03, ...,
        0.00000000e+00, 3.00000000e+00, 6.00000000e-01]])

In [90]:
# We combine our two tfidf (sparse) matrices and X_metadata
# ---
#
X = scipy.sparse.hstack([df_word_vect, df_char_vect,  X_metadata]).toarray()
X

array([[0.        , 0.        , 0.        , ..., 1.        , 2.        ,
        0.9       ],
       [0.        , 0.        , 0.        , ..., 3.        , 4.        ,
        0.7625    ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 2.        , 1.        ,
        0.83333333],
       [0.        , 0.        , 0.        , ..., 3.        , 0.        ,
        0.56785714],
       [0.        , 0.        , 0.        , ..., 0.        , 3.        ,
        0.6       ]])

In [41]:
# Getting our response variable
# ---
#
y = np.array(df.iloc[:, 0])
y

array([0, 4, 0, ..., 0, 4, 0])

### 4. Data Modelling

During this step, we will use machine learning algorithms to train and test our sentiment analysis models.

In [91]:
# Splitting our data
# ---
#
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [92]:
# Fitting our model
# ---
#

# Importing the algorithms
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression

nb_classifier = MultinomialNB() 
lr_classifier = LogisticRegression(max_iter=1000) 

# Training our model
nb_classifier.fit(X_train, y_train) 
lr_classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [93]:
# Making predictions
# ---
#
y_predict_nb = nb_classifier.predict(X_test) 
y_predict_lr = lr_classifier.predict(X_test)

In [94]:
# Evaluating the Models
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Accuracy scores
# ---
#
print("Naive Bayes Classifier:\n", accuracy_score(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", accuracy_score(y_test, y_predict_lr))

Naive Bayes Classifier:
 0.732
Logistic Regression Classifier: 
 0.759


In [95]:
# Confusion matrices
# ---
# 
print("Naive Bayes Classifier: \n", confusion_matrix(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", confusion_matrix(y_test, y_predict_lr))

Naive Bayes Classifier: 
 [[758 292]
 [244 706]]
Logistic Regression Classifier: 
 [[792 258]
 [224 726]]


In [96]:
# Classification Reports
# ---
#
print("Naive Bayes Classifier: \n", classification_report(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", classification_report(y_test, y_predict_lr))

Naive Bayes Classifier: 
               precision    recall  f1-score   support

           0       0.76      0.72      0.74      1050
           4       0.71      0.74      0.72       950

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000

Logistic Regression Classifier: 
               precision    recall  f1-score   support

           0       0.78      0.75      0.77      1050
           4       0.74      0.76      0.75       950

    accuracy                           0.76      2000
   macro avg       0.76      0.76      0.76      2000
weighted avg       0.76      0.76      0.76      2000



**Evaluation our Models**

* **Accuracy:** the percentage of texts that were assigned the correct topic.
* **Precision:** the percentage of texts the classifier classified correctly out of the total number of texts it predicted for each topic
* **Recall:** the percentage of texts the model predicted for each topic out of the total number of texts it should have predicted for that topic.
* **F1 Score:** the average of both precision and recall.

To improve our model, we can try perfoming other text processing techniques that would better prepare our data for fitting our model. We can also use different vectorizing techniques, implement other machine learning models and perform hyperparameter tuning.

### 5. Recommendations


Our best model had an accuracy of 73.25% and use it for classifying newer tweets. We can improve this performance by performing hyperparameter tuning and feature engineering methods. 